In [21]:
import pandas as pd
import json

In [16]:
schools = pd.read_json('https://stem-mentoring.firebaseio.com/school.json')
cities = pd.read_json('https://stem-mentoring.firebaseio.com/cities.json')
students = pd.read_json('https://stem-mentoring.firebaseio.com/students.json').T
confirm = pd.read_json('https://stem-mentoring.firebaseio.com/confirm.json').T

In [20]:
students.school.map(lambda x:json.loads(x))

-KJhoiAS6gSOwK88N8I5    {'value': '민족사관고등학교', 'city': 116, 'label': '민...
-KJhyVFWtRFSeyX6dQSN    {'value': '삼천포고등학교', 'city': 220, 'label': '삼천...
-KJjUTe8dw4VYTH6BKox    {'value': '분당대진고등학교', 'city': 89, 'label': '분당...
-KJkhkp_AspAs-T45btZ    {'value': '안양외국어고등학교', 'city': 92, 'label': '안...
-KK4NO6eWu0VfJJHwSI4    {'value': '서울과학고등학교', 'city': 8, 'label': '서울과...
-KK4TVE6AYraXEIrljzk    {'value': '대원외국어고등학교', 'city': 6, 'label': '대원...
-KK4W3jhhEOOrzcWFnb5    {'value': '서울과학고등학교', 'city': 8, 'label': '서울과...
-KK4X7ljrrT2t2xbfIBV    {'value': '서울과학고등학교', 'city': 8, 'label': '서울과...
-KK4YzhzayPCuXCBhYfg    {'value': '서울고등학교', 'city': 18, 'label': '서울고등...
-KKZYpNQoRqQ28tLFHT_    {'value': '경남과학고등학교', 'city': 216, 'label': '경...
Name: school, dtype: object